# DATA20001 Deep Learning - Group Project
## Image project

**Due Thursday, May 20, before 23:59.**

The task is to learn to assign the correct labels to a set of images.  The images are originally from a photo-sharing site and released under Creative Commons-licenses allowing sharing.  The training set contains 20 000 images. We have resized them and cropped them to 128x128 to make the task a bit more manageable.

We're only giving you the code for downloading the data. The rest you'll have to do yourselves.

Some comments and hints particular to the image project:

- One image may belong to many classes in this problem, i.e., it's a multi-label classification problem. In fact there are images that don't belong to any of our classes, and you should also be able to handle these correctly. Pay careful attention to how you design the outputs of the network (e.g., what activation to use) and what loss function should be used.

- As the dataset is pretty imbalanced, don't focus too strictly on the outputs being probabilistic. (Meaning that the right threshold for selecting the label might not be 0.5.)

- Image files can be loaded as numpy matrices for example using `imread` from `matplotlib.pyplot`. Most images are color, but a few grayscale. You need to handle the grayscale ones somehow as they would have a different number of color channels (depth) than the color ones.

- In the exercises we used e.g., `torchvision.datasets.MNIST` to handle the loading of the data in suitable batches. Here, you need to handle the dataloading yourself.  The easiest way is probably to create a custom `Dataset`. [See for example here for a tutorial](https://github.com/utkuozbulak/pytorch-custom-dataset-examples).

## Get the data

In [45]:
import sys
import os
from os.path import join
from os.path import abspath
from os.path import split

import torch
import torchvision
from torchvision.datasets.utils import download_url
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import zipfile

import matplotlib.pyplot as plt

root_dir = os.getcwd()
if root_dir not in sys.path:
    sys.path.append(root_dir)
    
train_path = 'train'

train_path_images = os.path.join(train_path, 'images')
train_path_annotations = os.path.join(train_path, 'annotations')

# check if we need to unzip data
try:
    os.listdir(train_path_images)
except OSError as e:
    data_folder_name = 'image-training-corpus+annotations'
    DATA_FOLDER_DIR = os.path.abspath(os.path.join(root_dir, data_folder_name))

    data_zip_name = 'dl2018-image-proj.zip'
    DATA_ZIP_DIR = os.path.abspath(os.path.join(DATA_FOLDER_DIR, data_zip_name))

    with zipfile.ZipFile(DATA_ZIP_DIR) as zip_f:
        zip_f.extractall(train_path)
        


class CustomDataset(Dataset):
    def __init__(self, train_path_images, train_path_annotations):
        # store path to images to get images dynamically
        self.train_path_images = train_path_images
        self.length = len(os.listdir(train_path_images))
        
        # get annotations
        self.annotations = {i:'other' for i in range(1, self.length+1)} #{1: 'bird', 2: 'baby', 3: 'bird', ...}
        for annotation_file in os.listdir(train_path_annotations):
            category, _ = annotation_file.split('.')
            with open(os.path.join(train_path_annotations, annotation_file)) as f:
                for line in f:
                    self.annotations[int(line.replace('\n', ''))] = category

        
        # (2) One way to do it is define transforms individually
        # When you define the transforms it calls __init__() of the transform
        #self.center_crop = transforms.CenterCrop(100)
        #self.to_tensor = transforms.ToTensor()
        self.grayscale = transforms.Grayscale(num_output_channels=1)
        
    def __getitem__(self, index):
        index += 1 # image labels start with 1
        label = self.annotations[index]
        img = plt.imread(os.path.join(self.train_path_images, 'im{}.jpg'.format(index)))
        
        # When you call the transform for the second time it calls __call__() and applies the transform 
        #data = self.center_crop(data)
        #data = self.to_tensor(data)
        img = self.grayscale(img)
        
        return img, label

    def __len__(self):
        return len(self.annotations)

custom_dataset = CustomDataset(train_path_images, train_path_annotations)

### Train Model

In [46]:

# Define data loader
dataset_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                                batch_size=10,
                                                shuffle=False)

for images, labels in dataset_loader:
    # Feed the data to the model
    print(images, labels)

TypeError: img should be PIL Image. Got <class 'numpy.ndarray'>

In [42]:
custom_dataset.__getitem__(0)

(array([[[201, 196, 177],
         [ 78,  79,  65],
         [105, 105,  95],
         ...,
         [ 44,  44,  42],
         [ 30,  33,  26],
         [ 35,  39,  25]],
 
        [[233, 229, 218],
         [113, 113, 101],
         [ 87,  87,  75],
         ...,
         [ 84,  84,  84],
         [ 81,  83,  80],
         [109, 112, 103]],
 
        [[245, 242, 223],
         [141, 142, 124],
         [ 77,  77,  65],
         ...,
         [121, 122, 117],
         [129, 129, 127],
         [103, 105, 100]],
 
        ...,
 
        [[ 34,  35,  29],
         [ 39,  40,  34],
         [ 46,  45,  40],
         ...,
         [ 21,  23,  18],
         [ 16,  19,  12],
         [ 12,  17,  10]],
 
        [[ 45,  44,  39],
         [ 37,  36,  31],
         [ 54,  54,  46],
         ...,
         [ 14,  16,  11],
         [  8,  11,   4],
         [  7,  12,   5]],
 
        [[ 49,  48,  43],
         [ 69,  69,  61],
         [ 71,  71,  61],
         ...,
         [  5,   7,   2],
  

The above command downloaded and extracted the data files into the `train` subdirectory.

The images can be found in `train/images`, and are named as `im1.jpg`, `im2.jpg` and so on until `im20000.jpg`.

The class labels, or annotations, can be found in `train/annotations` as `CLASSNAME.txt`, where CLASSNAME is one of the fourteen classes: *baby, bird, car, clouds, dog, female, flower, male, night, people, portrait, river, sea,* and *tree*.

Each annotation file is a simple text file that lists the images that depict that class, one per line. The images are listed with their number, not the full filename. For example `5969` refers to the image `im5969.jpg`.

## Your stuff goes here ...

In [ ]:
# Here goes your code ...

## Save your model

It might be useful to save your model if you want to continue your work later, or use it for inference later.

In [ ]:
torch.save(model.state_dict(), 'model.pkl')

The model file should now be visible in the "Home" screen of the jupyter notebooks interface.  There you should be able to select it and press "download".  [See more here on how to load the model back](https://github.com/pytorch/pytorch/blob/761d6799beb3afa03657a71776412a2171ee7533/docs/source/notes/serialization.rst) if you want to continue training later.

## Download test set

The testset will be made available during the last week before the deadline and can be downloaded in the same way as the training set.

## Predict for test set

You should return your predictions for the test set in a plain text file.  The text file contains one row for each test set image.  Each row contains a binary prediction for each label (separated by a single space), 1 if it's present in the image, and 0 if not. The order of the labels is as follows (alphabetic order of the label names):

    baby bird car clouds dog female flower male night people portrait river sea tree

An example row could like like this if your system predicts the presense of a bird and clouds:

    0 1 0 1 0 0 0 0 0 0 0 0 0 0
    
The order of the rows should be according to the numeric order of the image numbers.  In the test set, this means that the first row refers to image `im20001.jpg`, the second to `im20002.jpg`, and so on.

If you have the prediction output matrix prepared in `y` you can use the following function to save it to a text file.

In [ ]:
np.savetxt('results.txt', y, fmt='%d')